In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import transformers

print(transformers.__version__)

4.37.2


In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
model_checkpoint = "VietAI/vit5-base-vietnews-summarization"

In [4]:
from torch.utils.data import Dataset, DataLoader

In [13]:
from datasets import load_dataset
from evaluate import load

raw_datasets =  load_dataset("toanduc/10k-final")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Original Text', 'Summary'],
        num_rows: 11585
    })
    validation: Dataset({
        features: ['Original Text', 'Summary'],
        num_rows: 907
    })
})

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [8]:
from transformers import BertTokenizer,BertModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [9]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = "summarize "

In [ ]:
import re
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [10]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Original Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/11585 [00:00<?, ? examples/s]

Map:   0%|          | 0/907 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
# batch_size = 4
model_name = model_checkpoint.split("/")[-1]
fine_tuned_model_name = f"{model_name}-sport"

In [20]:
metric = load("rouge")

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [24]:
num_epochs = 16
batch_size = 4
args = Seq2SeqTrainingArguments(
  fine_tuned_model_name,
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=num_epochs,
  predict_with_generate=True,
  fp16=True,
)


trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.793100,1.624598,25.190200,11.984200,18.712500,21.876500,19.000000
2,1.586900,1.555714,24.810400,12.918000,19.186700,21.909100,19.000000
3,1.456800,1.526419,24.349800,12.645200,18.742600,21.474800,19.000000
4,1.337500,1.525925,24.358400,12.284300,18.717700,21.429600,19.000000
5,1.241700,1.529901,24.363300,11.990100,18.557300,21.454100,19.000000
6,1.170500,1.553259,24.431400,12.333700,18.751700,21.566900,19.000000
7,1.096500,1.571607,24.756200,12.488300,18.892600,21.728100,19.000000
8,1.047900,1.591844,24.346400,12.401800,18.672500,21.389100,19.000000
9,0.999300,1.611639,24.625400,12.121300,18.709100,21.574700,19.000000
10,0.951200,1.642732,24.613300,12.408800,18.824400,21.616200,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=46352, training_loss=1.1007067535842827, metrics={'train_runtime': 30402.709, 'train_samples_per_second': 6.097, 'train_steps_per_second': 1.525, 'total_flos': 1.5672533477465088e+17, 'train_loss': 1.1007067535842827, 'epoch': 16.0})

In [ ]:
trainer.push_to_hub()

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

events.out.tfevents.1710155169.50451a3d7017.735.0:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]